# End-to-End Data Analysis Project on Sales Dataset

<p style="text-align: justify;">Data analysis is the systematic process of analyzing, cleaning, manipulating, and modeling data with the aim of finding relevant information, coming to conclusions and assisting in decision-making.Python is widely used for data science because of its flexibility of use, adaptability, and significant library support. It allows for easy data cleaning, exploration, visualization, and modeling, making it an important tool for making intelligent decisions and extracting insights from complex data. Power BI is a business intelligence tool that provides interactive data visualization and analysis capabilities to help organizations make informed decisions and drive business success. I first imported data into MySQL Workbench in this project and connected it with Jupyter notebook. Here, I use python for data collecting, wrangling, manipulating, and making insights according to the problem statements. After That, I make an interactive dashboard using Power BI.</p>

# Contents
<ol>
    <li>Problem Statements</li>
    <li>Data Collection</li>
    <li>Data Cleaning and Pre-processing</li>
    <li>EDA perform according to the problem statement</li>        
        <ol type="a">
            <div>4.1 Which days of the week do customers use to buy more?</div>
            <div>4.2 Top of the most selling products by category.</div>
            <div>4.3 Whose are our top customers by revenue?</div>
            <div>4.4 Which are our top returning customers?</div>
            <div>4.5 Which type of customers generated the most revenue?</div>
            <div>4.6 Which market is most well profitable?</div>
            <div>4.7 Which zone is most well profitable?</div>
            <div>4.8 Which of our company's products don't do well on the market?</div>            
        </ol>
    <li>Power Bi Dashboard</li>
    <li>Conclusions</li>
    <li>Reccomendations & Next Steps</li>
    <li>Resources</li>
</ol>  

# 1. Problem Statements <a class="anchor" id="chapter1"></a>

<p style="text-align: justify;">This case study is based on a computer hardware company that is struggling in a market that is constantly changing. The head of sales chooses to invest in a data analysis project and wants to create a Power BI dashboard that would provide him with real-time sales insights.</p>
<p>So, here are some questions that I'll try to provide insights into by answering these questions.</p>
<ol>
    <li>Which days of the week do customers use to buy more?</li>
    <li>Top of the most selling products by category.</li>
    <li>Whose are our top customers by revenue?</li>
    <li>Which are our top returning customers?</li>
    <li>Which type of customers generated the most revenue?</li>
    <li>Which market is most profitable?</li>
    <li>Which zone is most well profitable?</li>
    <li>Which of our company's products don't do well on the market?</li>    
</ol>

# 2. Data Collection <a class="anchor" id="chapter2"></a>
<p style="text-align: justify;">I took this data from the Codebasics. They provided real sales data in SQL format which belongs to Atliq Technologies. I have attached the link below. I linked this Jupyter Notebook to the MySQL Workbench after importing the .sql file to complete all of the data analysis steps.</p>
<p>Before starting, I have to import essential libraries.</p>

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import mysql.connector as sql
import warnings
from IPython.core.display import display, HTML
from IPython.display import IFrame
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)


warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [55]:
# Creates a connection to a SQL database using the mysql library.
db_connection = sql.connect(host='localhost', database='sales', user='Abdun', password='1234')

In [56]:
"""
This code retrieves data from a MySQL database using the pandas library 
and stores it in variables as a data frame for further analysis.
"""

customers = pd.read_sql('SELECT * FROM customers', con=db_connection)
date = pd.read_sql('SELECT * FROM date', con=db_connection)
markets = pd.read_sql('SELECT * FROM markets', con=db_connection)
products = pd.read_sql('SELECT * FROM products', con=db_connection)
transactions = pd.read_sql('SELECT * FROM transactions', con=db_connection)

In [57]:
for i in ['transactions', 'customers','date', 'markets', 'products']:
    display(HTML(f'<h4>{i.capitalize()}</h4>'))
    display(eval(i).sample(2))
    display(HTML(f"{i.capitalize()} table have {eval(i).shape[0]} columns and {eval(i).shape[1]} rows."))


,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency,profit_margin_percentage,profit_margin,cost_price
138264,Prod239,Cus005,Mark004,2019-03-28,8,384.0,INR,-0.12,-46.08,430.08
95679,Prod294,Cus005,Mark004,2018-06-06,3,426.0,INR,0.17,72.42,353.58


,customer_code,custmer_name,customer_type
19,Cus020,Nixon,E-Commerce
4,Cus005,Premium Stores,Brick & Mortar


,date,cy_date,year,month_name,date_yy_mmm
433,2018-08-08,2018-08-01,2018,August,18-Aug\r
656,2019-03-19,2019-03-01,2019,March,19-Mar\r


,markets_code,markets_name,zone
4,Mark005,Kanpur,North
5,Mark006,Bengaluru,South


,product_code,product_type
252,Prod253,Own Brand\r
141,Prod142,Own Brand\r


# 3. Data Cleaning and Pre-processing <a class="anchor" id="chapter2"></a>

<p style="text-align: justify;">Data cleaning and pre-processing are crucial steps in the data analysis process that help ensure the quality and reliability of the data. They involve cleaning and transforming raw data into a usable format, correcting errors and inconsistencies, and converting data into a standard format. By performing data cleaning and pre-processing, we can improve the accuracy of our analysis and make informed decisions based on trustworthy data. The first thing we'll do is verify each DataFrame to see if any data is missing. After that, we will remove all noises from the data, and then convert every feature into a suitable data type.</p>

In [58]:
def missing_values_table(df, i):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print(f"Our selected data frame {str(i)} have {str(df.shape[1])} columns. There are {str(mis_val_table_ren_columns.shape[0])} columns that have missing values.")
    if mis_val.sum() == 0:
        return None
    else:
        return mis_val_table_ren_columns.T

In [59]:
for i in ['date', 'customers','products', 'markets', 'transactions']:
    display(HTML(f'<h4>{i.capitalize()}</h4>'))
    display(missing_values_table(eval(i), i))


Our selected data frame date have 5 columns. There are 0 columns that have missing values.


None

Our selected data frame customers have 3 columns. There are 0 columns that have missing values.


None

Our selected data frame products have 2 columns. There are 0 columns that have missing values.


None

Our selected data frame markets have 3 columns. There are 0 columns that have missing values.


None

Our selected data frame transactions have 10 columns. There are 0 columns that have missing values.


None

We saw that there are no missing values in our all data frames.
## 3.1 Perform data wrangling on markets dataframe.

<p style="text-align: justify;">The markets table has three columns and 17 rows. From this table we can see, Atliq sold products in three zones and 16 markets. Here we see that Atliq sold some products in New York and Paris Whereas, our analysis has to focus only on India's sales. For that reason, we will remove these two rows.</p>

In [60]:
display(markets.markets_name.value_counts().keys())
display(len(markets.markets_name.value_counts().keys()))

Index(['Bhopal', 'Chennai', 'Mumbai', 'Ahmedabad', 'Delhi NCR', 'Kanpur',
       'Bengaluru', 'Lucknow', 'Patna', 'Kochi', 'Nagpur', 'Surat',
       'Hyderabad', 'Bhubaneshwar', 'New York', 'Paris'],
      dtype='object')

16

In [61]:
markets.zone.value_counts().keys()

Index(['North', 'South', 'Central', ''], dtype='object')

In [62]:
display(transactions[transactions.market_code == 'Mark997'])
display(transactions[transactions.market_code == 'Mark999'])

,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency,profit_margin_percentage,profit_margin,cost_price


,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency,profit_margin_percentage,profit_margin,cost_price


In [63]:
markets = markets[markets.zone != '']
markets

,markets_code,markets_name,zone
0,Mark001,Chennai,South
1,Mark002,Mumbai,Central
2,Mark003,Ahmedabad,North
3,Mark004,Delhi NCR,North
4,Mark005,Kanpur,North
5,Mark006,Bengaluru,South
6,Mark007,Bhopal,Central
7,Mark008,Lucknow,North
8,Mark009,Patna,North
9,Mark010,Kochi,South


## 3.2 Perform data wrangling on date dataframe.

<p style="text-align: justify;">There are five features in the date table. however, three of them are completely superfluous. Therefore, we shall remove these columns. And we have to convert the other two columns into DateTime type. </p>

In [64]:
date.sample(2)

,date,cy_date,year,month_name,date_yy_mmm
701,2019-05-03,2019-05-01,2019,May,19-May\r
300,2018-03-28,2018-03-01,2018,March,18-Mar\r


In [65]:
date.drop(['year', 'month_name', 'date_yy_mmm'], axis=1, inplace=True)

In [66]:
date['date'] = pd.to_datetime(date.date) # string to datetime
date['cy_date'] = pd.to_datetime(date.cy_date) # string to datetime
date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1126 entries, 0 to 1125
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     1126 non-null   datetime64[ns]
 1   cy_date  1126 non-null   datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 17.7 KB


## 3.3 Perform data wrangling on customers dataframe.

<p style="text-align: justify;">
The customer's data frame has no issues. In this case, we have nothing to do.
</p>

In [67]:
customers.sample(3)

,customer_code,custmer_name,customer_type
17,Cus018,Electricalslance Stores,Brick & Mortar
7,Cus008,Acclaimed Stores,Brick & Mortar
29,Cus030,Synthetic,E-Commerce


In [68]:
customers.isnull().sum()

customer_code    0
custmer_name     0
customer_type    0
dtype: int64

In [69]:
customers.customer_type.value_counts().keys()

Index(['Brick & Mortar', 'E-Commerce'], dtype='object')

In [70]:
customers.customer_code.value_counts().keys()

Index(['Cus001', 'Cus029', 'Cus022', 'Cus023', 'Cus024', 'Cus025', 'Cus026',
       'Cus027', 'Cus028', 'Cus030', 'Cus002', 'Cus031', 'Cus032', 'Cus033',
       'Cus034', 'Cus035', 'Cus036', 'Cus037', 'Cus021', 'Cus020', 'Cus019',
       'Cus018', 'Cus003', 'Cus004', 'Cus005', 'Cus006', 'Cus007', 'Cus008',
       'Cus009', 'Cus010', 'Cus011', 'Cus012', 'Cus013', 'Cus014', 'Cus015',
       'Cus016', 'Cus017', 'Cus038'],
      dtype='object')

In [71]:
customers.custmer_name.value_counts().keys()

Index(['Surge Stores', 'Electricalsocity', 'Electricalslytical', 'Sound',
       'Power', 'Path', 'Insight', 'Control', 'Sage', 'Synthetic',
       'Nomad Stores', 'Zone', 'Elite', 'All-Out', 'Expression', 'Relief',
       'Novus', 'Propel', 'Modular', 'Nixon', 'Electricalsopedia Stores',
       'Electricalslance Stores', 'Excel Stores', 'Surface Stores',
       'Premium Stores', 'Electricalsara Stores', 'Info Stores',
       'Acclaimed Stores', 'Electricalsquipo Stores', 'Atlas Stores',
       'Flawless Stores', 'Integration Stores', 'Unity Stores',
       'Forward Stores', 'Electricalsbea Stores', 'Logic Stores',
       'Epic Stores', 'Leader'],
      dtype='object')

## 3.4 Perform data wrangling on products dataframe.

<p style="text-align: justify;">
   There are two columns in the product table with the same error in both columns. "/r" exists in every data. We'll remove it. 
</p>

In [72]:
products.product_type.value_counts().keys()

Index(['Own Brand\r', 'Distribution\r'], dtype='object')

In [73]:
products.product_type = products.product_type.str[:-1]

In [74]:
products.product_type.value_counts().keys()

Index(['Own Brand', 'Distribution'], dtype='object')

## 3.5 Perform data wrangling on transactions dataframe.

<p style="text-align: justify;">
The transaction data frame contains nine columns. There are first four columns represented as keys and the other columns provide the number of sales, amount of sales, currency, the profit margin in percentage, the profit margin, And cost prices. This data frame does not have any big issues except one, some transactions happen in US currency. So, we have to convert this amount into the Indian rupee.
</p>

In [75]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148395 entries, 0 to 148394
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   product_code              148395 non-null  object 
 1   customer_code             148395 non-null  object 
 2   market_code               148395 non-null  object 
 3   order_date                148395 non-null  object 
 4   sales_qty                 148395 non-null  int64  
 5   sales_amount              148395 non-null  float64
 6   currency                  148395 non-null  object 
 7   profit_margin_percentage  148395 non-null  float64
 8   profit_margin             148395 non-null  float64
 9   cost_price                148395 non-null  float64
dtypes: float64(4), int64(1), object(5)
memory usage: 11.3+ MB


In [76]:
transactions.currency.value_counts().keys()

Index(['INR', 'USD'], dtype='object')

In [77]:
transactions.describe()

,sales_qty,sales_amount,profit_margin_percentage,profit_margin,cost_price
count,148395.000000,1.483950e+05,148395.000000,148395.000000,1.483950e+05
mean,16.370376,6.636433e+03,0.024448,166.158350,6.470649e+03
std,115.394269,3.008649e+04,0.218956,6850.373158,2.977992e+04
min,1.000000,5.000000e+00,-0.350000,-369348.500000,3.050000e+00
25%,1.000000,1.760000e+02,-0.160000,-67.320000,1.665000e+02
50%,1.000000,5.190000e+02,0.020000,5.550000,5.082600e+02
75%,7.000000,3.065000e+03,0.210000,105.600000,2.907130e+03
max,14049.000000,1.510944e+06,0.400000,481775.040000,1.846742e+06


In [78]:
transactions[transactions.currency == 'USD']

,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency,profit_margin_percentage,profit_margin,cost_price
135937,Prod003,Cus005,Mark004,2017-11-20,59,500.0,USD,0.31,11625.0,25875.0
135938,Prod003,Cus005,Mark004,2017-11-22,36,250.0,USD,0.17,3187.5,15562.5


In [79]:
transactions['order_date'] = pd.to_datetime(transactions.order_date)

In [80]:
# Add a new column for normalize sales amount. The USD transactions sales convert into Indian Rupee.
transactions['norm_sales_amount'] = np.where(transactions.currency == 'USD', transactions.sales_amount*75, transactions.sales_amount)
transactions.loc[(transactions['currency'] == 'USD')]

,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency,profit_margin_percentage,profit_margin,cost_price,norm_sales_amount
135937,Prod003,Cus005,Mark004,2017-11-20,59,500.0,USD,0.31,11625.0,25875.0,37500.0
135938,Prod003,Cus005,Mark004,2017-11-22,36,250.0,USD,0.17,3187.5,15562.5,18750.0


## 3.6 Combining all Data Frames into a Single Data Frame.

In [81]:
df = transactions.merge(customers, on='customer_code', how='inner')
df = df.merge(markets, left_on='market_code', right_on='markets_code', how='inner')
df = df.merge(products, on='product_code', how='left')
df = df.merge(date, left_on='order_date', right_on='date', how='inner')

In [82]:
display(df.sample(2))
display(df.shape)

,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency,profit_margin_percentage,profit_margin,cost_price,norm_sales_amount,custmer_name,customer_type,markets_code,markets_name,zone,product_type,date,cy_date
46356,Prod106,Cus028,Mark003,2019-02-01,2,2259.0,INR,0.01,22.59,2236.41,2259.0,Sage,E-Commerce,Mark003,Ahmedabad,North,Own Brand,2019-02-01,2019-02-01
4767,Prod295,Cus005,Mark004,2018-07-09,1,162.0,INR,0.04,6.48,155.52,162.0,Premium Stores,Brick & Mortar,Mark004,Delhi NCR,North,NaN,2018-07-09,2018-07-01


(148395, 19)

# 4. EDA perform according to the problem statement

## 4.1 Which days of the week do customers use to buy more?

<p style="text-align: justify;">
Let's answer this question by observing the dataframe df. We will find out which day most of the orders we received or customers use to buy more.
</p>

In [83]:
days_of_week = df.order_date.dt.day_name().str[:3]
days_of_week.value_counts()

Fri    34803
Thu    29265
Tue    29117
Wed    28724
Mon    25961
Sat      403
Sun      122
Name: order_date, dtype: int64

In [84]:
fig = px.bar(
    y = days_of_week.value_counts(), 
    x = days_of_week.value_counts().keys(),    
    color_discrete_sequence=px.colors.qualitative.Antique
)

fig.update_layout(
    autosize=False,
    width=500,
    height=350,
)
colors = px.colors.diverging.Spectral

fig.update_traces(marker_color=colors)


fig.show()

* As we see that Sat and Sun are very lowest order we recieved.
* The highest orders came on Friday.

## 4.2 Top of the most selling products by category.

<p style="text-align: justify;">
We have to find out top-selling products based on category. However, there are two types of products own-brand and Distribution products. Here, we'll illustrate two bar charts in order to show the top 10 products in each group.
</p>

In [85]:
all_products = df[['product_code', 'sales_qty', 'product_type']].groupby(
    ['product_code', 'product_type']).aggregate(
        {'sales_qty': 'sum'}
)    
all_products = all_products.reset_index(level=['product_code', 'product_type'])
all_products.sample(3)

,product_code,product_type,sales_qty
243,Prod244,Own Brand,15136
172,Prod173,Distribution,67
52,Prod053,Own Brand,62147


In [86]:
top_Own_Brand = all_products[all_products['product_type'] == 'Own Brand'].nlargest(10, 'sales_qty')
top_Distribution = all_products[all_products['product_type'] == 'Distribution'].nlargest(10, 'sales_qty')

In [87]:
top_products = pd.concat([top_Distribution, top_Own_Brand])

In [88]:
fig = px.bar(
    data_frame=top_products, 
    x="product_code", 
    y="sales_qty", 
    color="product_type"
)

fig.show()

* Here we have seen the top ten products of both types.

## 4.3 Whose are our top customers by revenue?

In [89]:
top_customers = df[['custmer_name', 'norm_sales_amount']].groupby('custmer_name').sum().nlargest(10, 'norm_sales_amount')
top_customers = top_customers.reset_index(level=['custmer_name'])
top_customers.head(2)

,custmer_name,norm_sales_amount
0,Electricalsara Stores,413333588.0
1,Electricalslytical,49644189.0


In [90]:
fig = px.bar(
    data_frame=top_customers, 
    x="norm_sales_amount", 
    y="custmer_name", 
    title='Top Customers By Revenue'
)

fig.update_traces(marker_color=colors)
fig.show()

* There are the top ten customers by most revenue. 
* Electricalsara Stores have bought products from us for more than 400 million rupees.
* And other nine customers have bought bellow the 50 million.

## 4.4 Which are our top returning customers?

In [91]:
count_of_returning = df[['custmer_name', 'order_date']].groupby('custmer_name').nunique()
count_of_returning = count_of_returning.reset_index(level=['custmer_name'])
top_return_customers = count_of_returning.nlargest(10, 'order_date')

In [92]:
fig = px.bar(
    data_frame=top_return_customers, 
    x="order_date", 
    y="custmer_name", 
    title='Top Returning Customers',
)

fig.update_traces(marker_color=colors)
fig.show()

* The top ten customers buy more than six hundred from us frequently.

## 4.5 Which type of customers generated the most revenue?

In [93]:
Type_customer = df[['customer_type', 'norm_sales_amount']].groupby('customer_type').sum()
Type_customer

,norm_sales_amount
customer_type,
Brick & Mortar,744525338.0
E-Commerce,240343625.0


In [94]:
fig = px.pie(
    values=Type_customer['norm_sales_amount'], 
    names=Type_customer.index, 
    title='Revenue By Type of Customers'
)

fig.update_layout(
    autosize=False,
    height=350,
)

fig.show()

* Here we see our one-third revenue comes from e-commerce. In comparison, around 25 percentage revenue comes from Brick & Mortar.

## 4.6 Which market is most well profitable?

In [95]:
markets = df[['markets_name', 'profit_margin']].groupby('markets_name').sum()
markets = markets.reset_index(level=['markets_name']).sort_values('profit_margin', ascending=False)
markets.T

,5,10,0,2,11,8,4,12,13,6,3,9,7,1
markets_name,Delhi NCR,Mumbai,Ahmedabad,Bhopal,Nagpur,Kochi,Chennai,Patna,Surat,Hyderabad,Bhubaneshwar,Lucknow,Kanpur,Bengaluru
profit_margin,11953798.52,4872639.77,2848268.51,2287149.69,1412109.72,703289.99,300573.0,182260.78,126677.05,46957.22,35596.36,31161.21,-65878.45,-77534.96


In [96]:
fig = px.bar(
    data_frame=markets, 
    x="markets_name", 
    y="profit_margin", 
    title='Profitable Market'
)

fig.update_layout(
    autosize=False,
    height=500,
)

fig.update_traces(marker_color=colors)
fig.show()

* Our sales operation happened in 13 states.
* Most of the profit comes from Delhi NCR around 12 million. The other four markets generated profits between one to five million.
*  From Kochi to the other six markets (on the left) have generated under 1 million.
* More than 500,000 rupees were lost in Kanpur and Bengaluru.

## 4.7 Which zone is most well profitable?

In [97]:
zones = df[['zone', 'norm_sales_amount', 'profit_margin']].groupby('zone').sum()
zones

,norm_sales_amount,profit_margin
zone,,
Central,263720983.0,8571899.18
North,675588017.0,15076287.62
South,45559963.0,1008881.61


In [98]:
fig = px.pie(
    values=zones['norm_sales_amount'], 
    names=zones.index, 
    title='Profitable zone by revenue',    
)

fig.update_layout(
    autosize=False,
    height=350,
)

fig.show()

In [99]:
fig = px.pie(
    values=zones['profit_margin'], 
    names=zones.index, 
    title='Profitable zone by profit margin',    
)

fig.update_layout(
    autosize=False,
    height=350,
)

fig.show()

* Around 68 percent of revenue comes from North India. Although generated profit above 61 percent.
* The Central zone generates profit more than 34 percent. Whereas only around 26 percent of revenue generates it.
* Less than five percent of revenue and profit come From the south zone.

## 4.8 Which of our company's products don't do well on the market?

In [100]:
# loss_products = df[df['profit_margin'] <= 0][['product_code', 'profit_margin']].groupby('product_code').mean()
loss_products = df[['product_code', 'profit_margin']].groupby('product_code').mean().nsmallest(10, 'profit_margin')
loss_products = loss_products.reset_index()
loss_products

,product_code,profit_margin
0,Prod073,-369348.500000
1,Prod107,-60156.090000
2,Prod163,-57999.960000
3,Prod084,-37980.142000
4,Prod142,-20096.550000
5,Prod125,-17290.000000
6,Prod068,-9246.650000
7,Prod336,-8732.104211
8,Prod203,-8555.520000
9,Prod022,-8441.245000


In [104]:
fig = px.bar(
    data_frame=loss_products, 
    x="profit_margin", 
    y="product_code", 
)

fig.update_layout(
    autosize=False,
    height=500,
)

fig.update_traces(marker_color=colors)
fig.show(renderer="svg")

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


* There are the top ten loss products.

Let's Extract sales data in CSV format.

In [102]:
df.to_csv('marged_sales_data.csv', index=False)

# 5. Power Bi Dashboard
<p style="text-align: justify;">

</p>

In [103]:
IFrame(
    src="https://app.powerbi.com/view?r=eyJrIjoiYjAzMWZjMDctMjNjYy00NTQ0LThlMTctMzhkZmZiYzNjMGJjIiwidCI6IjZkMGIwZGQ1LTg2YmMtNGJhOC05NjMxLTZjZjM1ODA1M2I1YiIsImMiOjEwfQ%3D%3D",
    width="900", 
    height="540", 
)

# 6. Conclusions
<p style="text-align: justify;">
After conducting thorough data analysis, it can be concluded that there are some interesting things between the products and customers. Some insights we found for sales are very useful for improving sales and making more profits.All found insight is added below.
</p>
<ol>
    <li>Fewer than the 500 orders the company received on Saturday and Sunday. In contrast, The highest orders the company received were on Friday.</li>
    <li>The company's own brand products sold at lower in comparison to distribution-type products.</li>
    <li>Returning Customers were frequently buying products whereas the revenue was quite uneven.</li>
    <li>At Bangalore and Kanpur, they lost more than 500,000 rupees. Losses were imminent for Patna, Surat, Hyderabad, Bhubaneswar, and Lucknow. These four markets were struggling.</li>
    <li>The Central zone generates profit of more than 34 percent. Whereas only around 26 percent of revenue generates it.</li>
    <li>Around 68 percent of revenue comes from North India. Although generated profit above 61 percent.</li>
    <li>Less than five percent of revenue and profit come From the south zone.</li>
    <li>The company made a loss of more than 350k on product Prod073.</li>
    <li>There are few products which are continuously doing lose.</li>
</ol>

# 7. Reccomendations & Next Steps

<p style="text-align: justify;">
We suggest a number of activities to boost sales performance based on the findings of our study of the sales data. First, we advise starting a focused marketing campaign targeting clients who have previously made purchases but haven't done so in a while. In order to promote recurring purchases, this can be done through personalized email campaigns, marketing offers, or loyalty programs.
</p>

<p style="text-align: justify;">
Secondly, we suggest conducting a market analysis to identify the specific reasons for the underperformance. This will include an evaluation of the local market conditions, consumer demographics, and the competitive landscape. Optimizing the product mix to better meet the needs of these markets. This could include introducing new products that are more appealing to local consumers or adjusting prices to better reflect local economic conditions. The company may also need to consider customizing its marketing messages to better resonate with the cultural preferences of these markets.

</p>
<p style="text-align: justify;">
Finally, we recommend implementing a sales performance tracking system to monitor progress against sales targets and KPIs. This will enable the sales team to identify areas for improvement and make adjustments to their strategy in real time.
</p>

<p style="text-align: justify;">
Overall, by implementing these recommendations, we believe that the company can improve its sales performance in the underperforming markets and achieve greater overall success across all 13 states.
</p>


## 8. Resources
1. https://github.com/codebasics/DataAnalysisProjects/tree/master/1_SalesInsights
2. https://www.youtube.com/playlist?list=PLeo1K3hjS3uva8pk1FI3iK9kCOKQdz1I9
3. https://www.sqlbi.com/p/introducing-dax-video-course/
4. https://www.sqlbi.com/p/introduction-to-data-modeling-for-power-bi-video-course/